In [1]:
import pandas as pd
import datetime

In [2]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

date_str = '2018-05-31'

print("Date: " + date_str)

Date: 2018-05-31


In [3]:
# Read the Calendar extract to get additional details
cal_1 = pd.read_json('/Users/taj/GitHub/scraping/stayz/WebData/nsw_calendar/stayz_calendar_' + date_str + '.json.zip')



cal_2 = cal_1[cal_1['property_id'].notnull()]

cal_2.set_index('property_id',inplace=True)


cal = cal_2

cal.head()

,calendar,ext_at,features,photos,review_count,review_value
property_id,,,,,,
9191650,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-31 09:19:10,Air conditioning\nBalcony\nBooks\nChild's high...,22,25,5.0
9067504,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-31 09:19:12,Air conditioning\nBooks\nChild's high chair\nC...,23,38,5.0
9140176,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-31 09:19:14,Balcony\nChild's high chair\nChildren welcome\...,23,52,5.0
9060221,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-31 09:19:16,Balcony\nBooks\nChild's high chair\nChildren w...,11,33,4.9
9050878,"<thead><tr><th scope=""col""><span title=""Monday...",2018-05-31 09:19:20,,14,73,4.7


In [4]:
position_dict = {}
position_index = 0


data_dict = {}


# Step one - Create the dictionary of all possible feature items
for index, row in cal.iterrows():
    
    a = row['features']
    
    #print("Features: " + a)
    #save_features = [0] * len(position_dict)
    
    b = a.split('\n')
    
    for b1 in b:
        # Check if it exists in the dictionary
        
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                #save_features[b1_pos] = 1

            else:
                position_dict[b1] = position_index

                b1_pos = position_index

                position_index += 1
            
print(position_dict)



{'Air conditioning': 0, 'Balcony': 1, 'Books': 2, "Child's high chair": 3, 'Children welcome': 4, 'Coffee maker': 5, 'Dining': 6, 'Dishes and utensils': 7, 'Dishwasher': 8, 'Games': 9, 'Garage': 10, 'Internet access': 11, 'Iron and board': 12, 'Kitchen': 13, 'Lawn/Garden': 14, 'Linens provided': 15, 'Living room': 16, 'Microwave': 17, 'Non-Smoking': 18, 'Outdoor grill': 19, 'Oven': 20, 'Parking': 21, 'Private pool': 22, 'Refrigerator': 23, 'Satellite/Cable': 24, 'Stove': 25, 'Toaster': 26, 'Towels provided': 27, 'Washing machine': 28, 'Clothes dryer': 29, 'Deck/Patio': 30, 'DVD player': 31, 'Hair dryer': 32, 'Heating': 33, 'Stereo': 34, 'Television': 35, 'Tennis': 36, 'Toys': 37, 'Video library': 38, 'Fireplace': 39, 'Games room': 40, 'Pets allowed': 41, 'Wheelchair accessible': 42, 'Telephone': 43, 'Video Games': 44, 'Long-term renters welcome': 45, 'Wood stove': 46, 'Ceiling fan': 47, 'Fitness room/Equipment': 48, 'Hot tub': 49, 'Heated pool': 50, 'Events allowed': 51, 'Communal pool

In [5]:

# Step 2 - scan and create the data
for index, row in cal.iterrows():
    #print(index)
    a = row['features']
    b = a.split('\n')
    
    save_features = [0] * len(position_dict)
    
    for b1 in b:    
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                save_features[b1_pos] = 1
            else:
                print("ERROR: found a new feature not in dictionary: '" + b1 + "'")
    #print("Index: " + index)
    #if ( index > 0 ):
    data_dict[index] = save_features
    
df1 = pd.DataFrame.from_dict(data_dict)


df2 = df1.T

col_n_1 = position_dict.keys()

col_n_2 = []


for c in col_n_1:
    # Cleaning text
    for char in '\'/-.,\n ':
        c=c.replace(char,'_')

    # All words lower case
    c = c.lower()
    
    col_n_2.append('f_' + c)

df2.columns = col_n_2

df2.head()

,f_air_conditioning,f_balcony,f_books,f_child_s_high_chair,f_children_welcome,f_coffee_maker,f_dining,f_dishes_and_utensils,f_dishwasher,f_games,...,f_hot_tub,f_heated_pool,f_events_allowed,f_communal_pool,f_indoor_pool,f_sauna,f_pantry_items,f_cot,f_eco_friendly,f_free_wifi
9191650,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9067504,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9140176,0,1,0,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9060221,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
9050878,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Write out the file to CSV so it can be joined in the main branch

csv_out = df2.to_csv()

fh = open('/Users/taj/GitHub/scraping/stayz/WebData/nsw_features/stayz_nsw_features_' + date_str + '.csv', 'w')

fh.write(csv_out)

fh.close()